In [1]:
!pip3 install --upgrade auto-gptq
!pip3 install --upgrade transformers optimum
!pip install paddlepaddle-gpu
!pip install paddleocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: 

In [49]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from paddleocr import PaddleOCR
import os
import re
import json

In [4]:
# def load_model():
model_basename = "model"

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# model_name_or_path="TheBloke/Mistral-7B-v0.1-AWQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                            device_map='cuda',
                                            #  fuse_layers=True,
                                            use_safetensors=True,
                                            trust_remote_code=True,
                                            #  inject_fused_attention=False,
                                              revision="gptq-4bit-32g-actorder_True",
                                            )

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [5]:
def extract_document_text(document_path):
  ocr_model = PaddleOCR(lang='en',use_gpu=True)
  img_path = document_path
  result = ocr_model.ocr(img_path)[0]
  texts = [res[1][0] for res in result]
  combined_text = '\n'.join(texts)
  return combined_text

In [46]:
def create_prompt(combined_text):
  prompt = f"""
  Extract Invoice Number, Client ID, and Date from the provided invoice text, ensuring that the 'Date' is a single value.The output should be in json format

  Receipt Text:
  {combined_text}

  """

  prompt_template=f'''<s>[INST] {prompt} [/INST] '''
  return prompt_template

In [62]:
def get_data(input_document_path):
  for file in os.listdir(input_document_path):
    file_path = os.path.join(input_document_path,file)
    input_text = extract_document_text(file_path)
    prompt_template = create_prompt(input_text)
    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.1, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512,repetition_penalty=1.1)
    out_put_json=(tokenizer.decode(output[0]))
    pattern = re.compile(r'\[INST\]((?:(?!<\/s>).)*)<\/s>', re.DOTALL)
    json_match = pattern.search(out_put_json)
    if json_match:
        invoice_data = json_match.group(1).strip()
        invoice_json = json.loads(invoice_data)
        print(json.dumps(invoice_json, indent=2))
    else:
        print("No match found.")

In [ ]:
get_data('/content/files')